#SQL Analytics On Delta Live Tables<br/>

1. **Application           :** Social Media Analytics <br/>
2. **Usecase               :** Performing SQL analytics and Machine Learning algorithms on Twitter data coming incrementally from data lake.<br/>
3. **Notebook Summary      :** This notebook is a part of social media analytics application which perform `SQL Analytics`.<br/>
4. **Notebook Description  :** Performing SQL Analytics on top of Silver layer to derive `Gold layer`, which has aggregated/curated data for visuals and dashboards.


###Feature List
1. Lakehouse Architecture
2. Batch processing Twitter Messages
3. Delta Live Tables (Bronze, Silver and Gold and aggregation)
4. Hashtag Count
5. Z-Order Optimization 
6. Table Caching
7. Time Travel

In [0]:
%pip install geopy folium

In [0]:
import requests
import base64
import mimetypes

def url_image_to_base64_type_tuple(img_url):
  """
  Provide a URL to an image and have it converted to base64.
  - Url e.g.: `https://databricks.com/wp-content/themes/databricks/assets/images/header_logo_2x.png`
  Returns tuple of base64 data, mime_type
  """
  # notice the additional decode('utf-8') call
  response = requests.get(img_url)
  b64_data = str(base64.b64encode(response.content).decode('utf-8'))
  return b64_data, response.headers['Content-Type']

def display_img(base64_data, mime_type, width_percent=None):
  """
  Use `displayHTML` to display the base64 data using mime type).
  - If width_percent is provided, will set as attribute on `img` element.
  """
  if width_percent is None:
    displayHTML(f"""<img src="data:{mime_type};base64,{base64_data}">""") 
  else:
    displayHTML(f"""<img src="data:{mime_type};base64,{base64_data}" width="{width_percent}%">""") 
    


#### Raw Twitter Data - Bronze
Our bronze layer stores the raw, unprocessed data from our Twitter API pulls. By leaving it in its raw state, we give ourselves the option to reprocess it for different purposes in the future. Thanks to Azure Data Lake Gen 2, we can maintain this data for as long as we need it at very low costs. The bronze layer is usually the domain of *data engineers* who then build pipelines to refine this data forward into the silver layer.

In [0]:
%sql
Select Time, Hashtag, RetweetCount, FavouriteCount, IsRetweet, HourOfDay, Language 
from lakedb.bronze_twitter_historical_data

 

In [0]:
%sql
Select Time, Hashtag, RetweetCount, FavouriteCount, IsRetweet, HourOfDay, Language  
from lakedb.bronze_twitter_historical_data
Where City <> 'Online'

### Filtered Twitter Data - Silver
In our silver layer, we've curated our raw Twitter data into something more usable for *data scientists.* They can take these cleaned up, detailed level tables and develop features for machine learning models as well as aggregated analytical datasets for data analysts.

In [0]:
%sql
Select Time, Hashtag, RetweetCount, FavouriteCount, IsRetweet, HourOfDay, Language
from lakedb.silver_twitter_historical_data

#### Curated Twitter Data - Gold

In our gold layer, we can enhance and refine our silver data sets even further into fit-for-purpose tables and views for specific analytical needs. Here we've augmented our Twitter data with a machine learning model identifying the sentiment (positive, neutral, or negative) of each Tweet so we can get a sense of how the Twitter tags we're analyzing are being used.

In [0]:
%sql
Select MLSentiment,Time, Hashtag, RetweetCount, FavouriteCount, IsRetweet, HourOfDay, Language  
from lakedb.gold_twitter_historical_data

#### Aggregations

By pre-emptively aggregating our data that rarely or slowly changes, we can provide a great performance benefit for our end users. Our DLT pipeline performs this aggregation of hashtag counts by the geolocation of the Tweets. By only updating this each time we ingest more Tweets, we can keep the aggregation table up to date and then quickly consume and visualize it in tools like Power BI.

One nice feature of Databricks notebooks is if a cell produces a DataFrame output (like the one below), you can also profile the data as well as generate quick visualizations. Throw in Markdown and comments and notebooks are a super convenient way to collaborate and communicate with your team, leadership, customers, and other stakeholders.

In [0]:
%sql
SELECT Hashtag, Count from lakedb.gold_twitter_historical_city_hashtagcount_data where city is not null order by count desc limit 10

One way to analyze our Twitter data is by geolocation. We can easily plot our Twitter hashtag counts from above on a fully interactive map visual - directly within our notebook! We could then share this notebook with others so they can explore the location and hashtag data in more detail.

In [0]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='twitter-analysis-cl')
#from pyspark.sql.functions import udf

def lon_lat(city):
    try:
      lon = geolocator.geocode(city)[1][1]
      lat = geolocator.geocode(city)[1][0]
      return lat, lon
    except:
        return 51.5073219, -0.1276474

In [0]:
df = spark.sql("SELECT * from lakedb.gold_twitter_historical_city_hashtagcount_data where city is not null order by count desc limit 10")
df = df.toPandas()
df = df.fillna("")
df['latitude'], df['longitude'] = zip(*df.apply(lambda x: lon_lat(x['city']), axis = 1)) #Looking up each city can take awhile! :)

In [0]:
hashtagcountdf=spark.createDataFrame(df) 
hashtagcountdf.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("lakedb.gold_twitter_historical_city_hashtag_count_loc")

In [0]:
%sql
SELECT hashtag, count, latitude, longitude from lakedb.gold_twitter_historical_city_hashtag_count_loc

#### Create world map - Twitter Messages
Map created below shows the location from where people were tweeting

In [0]:
# Import general useful packages
import numpy as np
import pandas as pd

# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster

# Empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)

# Creating map
for i in range(len(df)):
  lat = df.iloc[i]['latitude']
  lon = df.iloc[i]['longitude']
  radius=5
  popup_text = """Country : {}<br>"""
  popup_text = popup_text.format(df.iloc[i]['city'])
  folium.CircleMarker(location = [lat, lon], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)

#show the map
world_map

#### Z-Order Optimization
Z-Ordering is a technique to colocate related information in the same set of files. <br/> This co-locality is automatically used by Delta Lake on Databricks data-skipping algorithms. <br/> This behaviour dramatically reduces the amount of data that Delta Lake on Databricks needs to read

In [0]:
%sql 
OPTIMIZE lakedb.gold_twitter_historical_data ZORDER BY city

####Caching
Caching reduces scanning of the original files in future queries. It basically caches contents of a table in Apache Spark cache. If a query is cached, then a temp view is created for this query.

In [0]:
%sql 
CACHE SELECT * FROM lakedb.gold_twitter_historical_data

### Time Travel
In audit history above we can view the history of the different versions of the table and load and display any of those versions. <br/>
In the exercise below we have displayed data in a specific version

In [0]:
from pyspark.sql.functions import *
history = spark.sql("DESCRIBE HISTORY lakedb.`gold_twitter_historical_data`")
latest_version = history.selectExpr("max(version)").collect()
version = (latest_version[0][0])
print("Version:") 
print(version)
df = spark.read.format("delta").option("versionAsOf", version).table("lakedb.gold_twitter_historical_data")
print("Rows:") 
print(df.count())
display(df.select(col("Time"),col("MLSentiment"),col("Hashtag"),col("RetweetCount"),col("FavouriteCount"),col("IsRetweet"),col("HourOfDay"),col("Language")))